## Module Imports

In [ ]:
include("emulation_functions.jl")
using BenchmarkTools
using CSV
using JLD2
using Dates
using MultivariateStats
import MultivariateStats: reconstruct
using GaussianProcesses
using NCDatasets
using DataFrames
using ForwardDiff
using LinearAlgebra
using NamedArrays
using Plots
using Plots.PlotMeasures
using Optimization
using OptimizationOptimJL
using Statistics
using StatsPlots
using Distributions
using Turing
using .Threads

## Set File Directory
Set the `directory_of_files` variable below to the path leading to your folder containing all the data and emulator files.

In [ ]:
# directory where all files are located. If it is this working directory, set this to "./"
directory_of_files = "./"; 
p_names = (:flnr,:slatop,:dleaf,:dsladlai,:leaf_long,:s_vc,:bbbopt,:mbbopt,:smpsc,:smpso,:rholvis,:rholnir,:taulvis,:taulnir,:rhosvis,:rhosnir,:xl,:displar,:z0mr,:vcmaxha,:vcmaxhd,:jmaxha,:jmaxhd,:roota_par,:rootb_par,:grperc)
p_names_dis = (:flnr,:slatop,:leaf_long, :rootb_par)

D = length(p_names_dis)

## Load data 

In [ ]:
# Dictionaries to make this all neater
output_var_header_names = Dict{String, String}("GPP" => "GPP_NT_VUT_REF", 
                                               "NEE" => "NEE_VUT_REF", 
                                               "LH" => "LE_F_MDS", 
                                               "SH" => "H_F_MDS")
site_names = Dict{String, String}("US_Me2" => "fluxnet_US_Me2_2003_2007", 
                                  "US_Dk3" => "fluxnet_US_Dk3_2003_2007",
                                  "CA_TP4" => "fluxnet_CA_TP4_2003_2007",
                                  "US_Blo" => "fluxnet_US_Blo_2003_2007",
    );

## Load default parameters

In [ ]:
p_default=[0.0509,0.01,0.04,0.0006,1.5,20.72,10000,9,-255000,-66000,0.07,0.35,0.05,0.1,0.16,0.39,0.01,0.67,0.055,72000,200000,50000,200000,3,1.25,0.3]
elements = ["flnr","slatop","dleaf","dsladlai","leaf_long","s_vc","bbbopt","mbbopt","smpsc","smpso","rholvis","rholnir","taulvis",
            "taulnir","rhosvis","rhosnir","xl","displar","z0mr","vcmaxha","vcmaxhd","jmaxha","jmaxhd","roota_par","rootb_par","grperc"]
p_hard = (  flnr =      [0.04, 0.3],                         
            slatop =    [0.003, 0.03],
            dleaf =     [0.03, 0.3],
            dsladlai =  [0.0002, 0.0035],
            leaf_long = [1.0, 12.0],
            s_vc =      [16.0,32.0],
            bbbopt =    [16000.0,60000.0],
            mbbopt =    [4.5, 15],
            smpsc =     [-642000.0, -125000.0],
            smpso =     [-125000.0, -17500.0],
            rholvis =   [0.025, 0.25],
            rholnir =   [0.25,0.55],
            taulvis =   [0.005,0.20],
            taulnir =   [0.15,0.45],
            rhosvis =   [0.05,0.30],
            rhosnir =   [0.20,0.75],
            xl =        [-0.5,0.375],
            displar =   [0.6,0.85],
            z0mr =      [0.04,0.09],
            vcmaxha =   [45000.0,90000.0],
            vcmaxhd =   [198000.0, 202000.0],
            jmaxha =    [30000.0,65000.0],
            jmaxhd =    [198000.0, 202000.0],
            roota_par = [2.0,18.0],
            rootb_par = [0.5,6.0],
            grperc =    [0.125,0.375] );

p_hard = [getproperty(p_hard, pn) for pn in propertynames(p_hard)];
p_hard_l = mapreduce(permutedims, vcat, p_hard)[:, 1];
p_hard_u = mapreduce(permutedims, vcat, p_hard)[:, 2];
p_norm = (p_default.- p_hard_l) ./ (p_hard_u .- p_hard_l)

## NOTE: 
#### `output_of_interest`
+ "GPP"
+ "NEE"
+ "LH"
+ "SH"


In [ ]:
# == SPECIFY THESE PARAMETERS TO SELECT OUTPUT VAR, SITE EMULATOR, AND CALIBRATION SITE OF INTEREST ===
c_site1_to_calibrate_to = "US_Me2"
c_site2_to_calibrate_to = "US_Dk3"
c_site3_to_calibrate_to = "US_Blo"
c_site4_to_calibrate_to = "CA_TP4"
c_site5_to_calibrate_to = "US_NR1"
c_site6_to_calibrate_to = "CA_Qfo"
c_site7_to_calibrate_to = "CA_Obs"
c_site8_to_calibrate_to = "CA_TP1"
c_site9_to_calibrate_to = "CA_TP2"
c_site10_to_calibrate_to = "CA_TP3"

t_site1_to_predict_to = "CA_Qfo"
output_of_interest = "GPP"


In [ ]:
#site =  site_names[site_to_calibrate_to]
output = output_var_header_names[output_of_interest]
begin_date = "2003" # Begin date of fluxnet data file (in case this changes in the future)
end_date = "2007" # End data of fluxnet data file (in case this changes in the future)

In [ ]:
function calculate_10day_average(data0)
    num_years = 5  
    monthday=[10, 10,11,10,10,8,10,10,11,10,10,10,10,10,11,10,10,10,10,10,11,10,10,11,10,10,10,10,10,11,10,10,10,10,10,11];
    monthly_average = Vector{Float64}()
    end_day=0
    for year in 1:num_years
        start_day=end_day+1
        for month in 1:36
            end_day = start_day + monthday[month]-1
            temp=mean(filter(!isnan,data0[start_day:end_day,:]), dims=1)
            push!(monthly_average,temp[1])
            start_day=end_day+1
        end
    end
    println(typeof(monthly_average))
    println(size(monthly_average))
    return monthly_average
end

In [ ]:
data = CSV.read("$(directory_of_files)/fluxnet_$(c_site1_to_calibrate_to)_$(begin_date)_$(end_date)_QC.txt", DataFrame, header=1, delim=",")
y_site1 = Array(data[!, output])
id_1=findall(y_site1.<0)
y_site1[id_1].=0
data = CSV.read("$(directory_of_files)/fluxnet_$(c_site2_to_calibrate_to)_$(begin_date)_$(end_date)_QC.txt", DataFrame, header=1, delim=",")
y_site2 = Array(data[!, output])
id_2=findall(y_site2.<0)
y_site2[id_2].=0
data = CSV.read("$(directory_of_files)/fluxnet_$(c_site3_to_calibrate_to)_$(begin_date)_$(end_date)_QC.txt", DataFrame, header=1, delim=",")
y_site3 = Array(data[!, output])
id_3=findall(y_site3.<0)
y_site3[id_3].=0
data = CSV.read("$(directory_of_files)/fluxnet_$(c_site4_to_calibrate_to)_$(begin_date)_$(end_date)_QC.txt", DataFrame, header=1, delim=",")
y_site4 = Array(data[!, output]);
id_4=findall(y_site4.<0)
y_site4[id_4].=0
data = CSV.read("$(directory_of_files)/fluxnet_$(c_site5_to_calibrate_to)_$(begin_date)_$(end_date)_QC.txt", DataFrame, header=1, delim=",")
y_site5 = Array(data[!, output]);
id_5=findall(y_site5.<0)
y_site5[id_5].=0

In [ ]:
ensembles0 = CSV.read("./parameters_normalized_emulator_result_US_Me2_GPP_200_4par.txt", DataFrame; header = false, transpose=true);
GPP1 = Matrix(ensembles0)'*24*3600
ensembles_mGPP1 = mean(GPP1, dims=1)
GPP_Me2_be_200=GPP1[:,144]
plot(GPP_Me2_be_200)

In [ ]:
ensembles0 = CSV.read("./parameters_normalized_emulator_result_US_Blo_GPP_200_4par.txt", DataFrame; header = false, transpose=true);
GPP1 = Matrix(ensembles0)'*24*3600
ensembles_mGPP1 = mean(GPP1, dims=1)
GPP_Blo_be_200=GPP1[:,8]
plot(GPP_Blo_be_200)

In [ ]:
ensembles0 = CSV.read("./parameters_normalized_emulator_result_US_NR1_GPP_200_4par.txt", DataFrame; header = false, transpose=true);
GPP1 = Matrix(ensembles0)'*24*3600
ensembles_mGPP1 = mean(GPP1, dims=1)
GPP_NR1_be_200=GPP1[:,32]
plot(GPP_NR1_be_200)

In [ ]:
GPP_01 = calculate_10day_average(y_site1)
GPP_02 = calculate_10day_average(y_site2)
GPP_03 = calculate_10day_average(y_site3)
GPP_04 = calculate_10day_average(y_site4)
GPP_05 = calculate_10day_average(y_site5)

In [ ]:
a1=std(y_site1-GPP_Me2_be_200)
a3=std(y_site3-GPP_Blo_be_200)
a5=std(y_site5-GPP_NR1_be_200)
println([a1,a3,a5])

In [ ]:
iid_normal_noise1 = a1*randn(1825)
GPP_01_be=calculate_10day_average(GPP_Me2_be_200+iid_normal_noise1)
iid_normal_noise3 = a3*randn(1825)
GPP_03_be=calculate_10day_average(GPP_Blo_be_200+iid_normal_noise3)
iid_normal_noise5 = a5*randn(1825)
GPP_05_be=calculate_10day_average(GPP_NR1_be_200+iid_normal_noise5)

## Load emulator (all GPs used in reconstruction)

In [ ]:
emulator_filename1 = "$(directory_of_files)/emulator_$(output_of_interest)_$(c_site1_to_calibrate_to)_200_4par.jld2";
@load emulator_filename1 array_GP pca_fit μ_z σ_z
emulator_components1 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z);
emulator_filename3 = "$(directory_of_files)/emulator_$(output_of_interest)_$(c_site3_to_calibrate_to)_200_4par.jld2";
@load emulator_filename3 array_GP pca_fit μ_z σ_z
emulator_components3 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z);
emulator_filename5 = "$(directory_of_files)/emulator_$(output_of_interest)_$(c_site5_to_calibrate_to)_200_4par.jld2";
@load emulator_filename5 array_GP pca_fit μ_z σ_z
emulator_components5 = (GPs = array_GP, T_PCA = pca_fit, μ_z = μ_z, σ_z = σ_z);

## Bayesian Parameter Calibration

## Create probabilistic model

Our likelihood looks like linear regression and features discrepency terms $\beta_0$ and $\beta_1$ in addition to noise term $\epsilon$.

### Likelihood: 
$$y \sim \beta_1 GP(\theta) + \epsilon, \qquad $$
$$\epsilon \stackrel{iid}{\sim} N(0,\sigma^2)$$

for now we assume a known $\sigma$, but we could estimate this parameter as well. $y$ is any quantity of interest predicted by the GP, $\theta$ are the model parameters (from the ELM model, the set of 26 params)

### Priors: 

$$\theta \sim p(\theta)$$ 
$$p(\theta) = Normal(0.5,0.25)$$
$$\log(\sigma) \sim N(\log 1, ((\log 2)/2)^2)$$

And I chose Normal priors for the regression parameters and LogNormal for the standard deviation. 


Define Prior distributions

ZJ: I change the priors and define the prob_model for each site, as the emulators are different

### Site 1

In [ ]:
#try other priors and define the model (emulator) outside for final validation (sample observation & simulated observation)
function priors(num_params)
    return filldist( Truncated(Normal(0.5,0.25), 0,1), num_params)
end
model_1(θ,N_PCA)= calibration_emulator1(θ, emulator_components1, N_PCA);

Q(y)=mean(y)

In [ ]:
@model function prob_model_1(y, num_params, N_PCA)
    # Define the priors
    p ~ priors(num_params)
    β_1 ~ Uniform(0.9,1.1)
    σ ~ LogNormal(log(1), log(2)/2)
    θ = reshape(p, 1,length(p)) # GP is very picky about dimensions, so we need to reshape here
    m=model_1(θ,N_PCA)
    y ~ MvNormal(β_1*m, σ^2*I )
    return Q(m)
end 

In [ ]:
alg = NUTS(1000,0.65)
strategy = MCMCSerial()
N_param=4
N_sample =5000
N_chain = 5

model = prob_model_1(GPP_01, N_param, 12)
@time chain1 = Turing.sample(model, alg, strategy, N_sample, N_chain; progress=true)
chain1 = replacenames(chain1, Dict("p[$i]" => p_names_dis[i] for i in 1:D))
@save "US_Me2_GPP_fluxnet_nobeta0_5000_5_4par_180day.jld2" chain1

### Site 3

In [ ]:
#try other priors and define the model (emulator) outside for final validation (sample observation & simulated observation)
function priors(num_params)
    return filldist( Truncated(Normal(0.5,0.25), 0,1), num_params)
end
model_3(θ,N_PCA)= calibration_emulator1(θ, emulator_components3, N_PCA);

Q(y)=mean(y)

In [ ]:
@model function prob_model_3(y, num_params, N_PCA)
    # Define the priors
    p ~ priors(num_params)
    β_1 ~ Uniform(0.9,1.1)
    σ ~ LogNormal(log(1), log(2)/2)
    
    θ = reshape(p, 1,length(p)) # GP is very picky about dimensions, so we need to reshape here
    m=model_3(θ,N_PCA)
    y ~ MvNormal(β_1*m, σ^2*I )
    return Q(m)
end 

In [ ]:
alg = NUTS(1000,0.65)
strategy = MCMCSerial()
N_param=4
N_sample =5000
N_chain = 5

model = prob_model_3(GPP_03, N_param, 12)
@time chain3 = Turing.sample(model, alg, strategy, N_sample, N_chain; progress=true)
chain3 = replacenames(chain3, Dict("p[$i]" => p_names_dis[i] for i in 1:D))
@save "US_Blo_GPP_fluxnet_nobeta0_5000_5_4par_180day.jld2" chain3

### site 5

In [ ]:
#try other priors and define the model (emulator) outside for final validation (sample observation & simulated observation)
function priors(num_params)
    return filldist( Truncated(Normal(0.5,0.25), 0,1), num_params)
end
model_5(θ,N_PCA)= calibration_emulator1(θ, emulator_components5, N_PCA);

Q(y)=mean(y)

In [ ]:
@model function prob_model_5(y, num_params, N_PCA)
    # Define the priors
    p ~ priors(num_params)
    β_1 ~ Uniform(0.9,1.1)
    σ ~ LogNormal(log(1), log(2)/2)
    
    θ = reshape(p, 1,length(p)) # GP is very picky about dimensions, so we need to reshape here
    m=model_5(θ,N_PCA)
    y ~ MvNormal(β_1*m, σ^2*I )
    return Q(m)
end 

In [ ]:
alg = NUTS(1000,0.65)
strategy = MCMCSerial()
N_param=4
N_sample =5000
N_chain = 5

model = prob_model_5(GPP_05, N_param, 12)
@time chain5 = Turing.sample(model, alg, strategy, N_sample, N_chain; progress=true)
chain5 = replacenames(chain5, Dict("p[$i]" => p_names_dis[i] for i in 1:D))
@save "US_NR1_GPP_fluxnet_nobeta0_5000_5_4par_180day.jld2" chain5

## Posterior predictions / observing system simulation experiments (OSSEs)

OSSE parameter sampling from posterior distributions

In [ ]:
N_samp0 = 1000

In [ ]:
@load "US_Me2_GPP_fluxnet_nobeta0_5000_5_4par_180day.jld2"  chain1
achain1=Array(chain1)
@load "US_Blo_GPP_fluxnet_nobeta0_5000_5_4par_180day.jld2"  chain3
achain3=Array(chain3)
@load "US_NR1_GPP_fluxnet_nobeta0_5000_5_4par_180day.jld2" chain5
achain5=Array(chain5)

In [ ]:
p_samp1 = achain1[Turing.sample(1:size(achain1,1), N_samp0),:]
p_samp3 = achain3[Turing.sample(1:size(achain1,1), N_samp0),:]
p_samp5 = achain5[Turing.sample(1:size(achain1,1), N_samp0),:]

In [ ]:
function simulate_observations(N, samples, model,N_PCA)
    N_samp = size(samples,1)
    y_sim = zeros(N, N_samp)
    @threads for i = 1:N_samp
       y_sim[:,i] .= sample_observation(samples[i,:], model,N_PCA)
    end
    y_sim
end

In [ ]:
using Statistics
using Distributions

# Define the function to calculate the 95% confidence interval
function confidence_interval(data::Vector{Float64}, confidence_level::Float64=0.05)
    # Sample mean
    mean_value = mean(data)

    # Sample standard deviation
    std_dev = std(data)

    # Sample size
    n = length(data)

    # Standard error of the mean
    sem = std_dev / sqrt(n)

    # For the specified confidence interval, the critical value from the t-distribution
    # Degrees of freedom: n - 1
    df = n - 1
    alpha = 1 - confidence_level
    t_critical = quantile(TDist(df), 1 - alpha / 2)

    # Margin of error
    margin_of_error = t_critical * sem

    # Confidence interval
    #lower_bound = mean_value - margin_of_error
    #upper_bound = mean_value + margin_of_error
    lower_bound = quantile(data,0.025)
    upper_bound = quantile(data,0.975)
    
    

    return (lower_bound, upper_bound)
end

In [ ]:
# Define a function to calculate RMSE
function rmse(actual, predicted)
    n = length(actual)
    sqrt(sum((actual .- predicted).^2) / n)
end

In [ ]:
# Define a function to calculate RMSE, NRMSE, and PRMSE
function rmse(actual, predicted; method="percentage")
    n = length(actual)
    rmse_value = sqrt(sum((actual .- predicted).^2) / n)

    if method == "range"
        # Normalized by range (max - min)
        norm_rmse = rmse_value / (maximum(actual) - minimum(actual))
    elseif method == "percentage"
        # Normalized by mean (percentage RMSE)
        norm_rmse = (rmse_value / mean(actual)) 
    else
        error("Invalid method. Choose 'range' or 'percentage'.")
    end

    return norm_rmse
end

In [ ]:
function r_squared(actual, predicted)
    # 计算均方和
    ss_total = sum((actual .- mean(actual)).^2)
    ss_residual = sum((actual .- predicted).^2)
    r2 = 1 - (ss_residual / ss_total)
    return r2
end

In [ ]:
ELM_Me2 = Dataset("US_Me2_GPP_g0001_g1300_2003_2007_res05.nc")
t = ELM_Me2["time"]
time1=year.(t)+dayofyear.(t)/365;

In [ ]:
using GLMakie
using CairoMakie

In [ ]:
fig = Figure(size= (900, 600));

In [ ]:
function sample_observation(sample, model,N_PCA)  
    σ = sample[6] 
    β_1=sample[5]
    p = sample[1:4]
    θ = reshape(p, 1, length(p))
    m = model_1(θ,N_PCA)
    err=a1*randn(length(m))
    y = β_1*m +err
end

In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp1, model_1, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end

xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis
#l = fig[1, 1] = GridLayout()
ax1 = Axis(fig[1, 1], xlabel = "", ylabel = "US-Me2",title="US-Me2",xticks=(xvalues, xlabels))
CairoMakie.ylims!(ax1, -5, 15)
# Add plots

p2=band!(ax1, time1[15:10:1805], ci[:, 1], ci[:, 2], color = (:pink, 1))
# Add mean line
p4=lines!(ax1, time1[15:10:1805], GPP_01_be, color = :blue, linewidth = 4, label = "BE+noise")
p3=lines!(ax1, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth = 2, label = "mean")
# Add scatter plots
p5=GLMakie.scatter!(ax1, time1[15:10:1805], GPP_01,  markersize=4,color = :green, label = "fluxnet")
rmse_value = rmse(GPP_01, y_sim_1_1_m)
text!(ax1, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
r2 = cor(GPP_01, y_sim_1_1_m)

legend=Legend(fig,[p2,p3,p4,p5], ["95% interval","mean","best estimate","fluxnet"],tellheight = false, halign =:left, valign =:top, orientation = :horizontal, framevisible = false)
fig[4,2] = legend
legend.width = Relative(1.2)
fig[1,1]=ax1

In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp3, model_1, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end

xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis

ax2 = Axis(fig[2, 1], xlabel = "", ylabel = "US-Blo", title = "",xticks=(xvalues, xlabels))
CairoMakie.ylims!(ax2, -5, 15)

# Add shaded area
band!(ax2, time1[15:10:1805], ci[:, 1], ci[:, 2],  color = (:pink,1))
# Add mean line
lines!(ax2, time1[15:10:1805], GPP_01_be, color = :blue, linewidth = 4, label = "BE+noise")
lines!(ax2, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth = 2, label = "mean")
GLMakie.scatter!(ax2, time1[15:10:1805], GPP_01, markersize=4,color = :green, label = "")
rmse_value = rmse(GPP_01, y_sim_1_1_m)
text!(ax2, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[2,1]=ax2


In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp5, model_1, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end

xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis

ax3 = Axis(fig[3, 1], xlabel = "year", ylabel = "US-NR1", title = "")
CairoMakie.ylims!(ax3, -5, 15)

# Add shaded area
band!(ax3, time1[15:10:1805], ci[:, 1], ci[:, 2],  color = (:pink, 1))
# Add mean line
lines!(ax3, time1[15:10:1805], GPP_01_be, color = :blue, linewidth = 4, label = "BE+noise")
lines!(ax3, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth = 2, label = "mean")
GLMakie.scatter!(ax3, time1[15:10:1805], GPP_01, markersize=4,color = :green, label = "")
rmse_value = rmse(GPP_01, y_sim_1_1_m)
text!(ax3, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[3,1]=ax3


In [ ]:
function sample_observation(sample, model,N_PCA)  
    σ = sample[6] 
    β_1=sample[5]
    p = sample[1:4]
    θ = reshape(p, 1, length(p))
    m = model_3(θ,N_PCA)
    err=a3*randn(length(m))
    y = β_1*m +err
end

In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp1, model_3, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end
# Create figure and axis
xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

ax4 = Axis(fig[1, 2], xlabel = "", ylabel = "", title = "US-Blo",xticks=(xvalues, xlabels))
CairoMakie.ylims!(ax4, -5, 15)

# Add shaded area
band!(ax4, time1[15:10:1805], ci[:, 1], ci[:, 2], color = (:pink,1))
# Add mean line
lines!(ax4, time1[15:10:1805], GPP_03_be, color = :blue, linewidth = 4, label = "BE+noise")
lines!(ax4, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth =2, label = "mean")
GLMakie.scatter!(ax4, time1[15:10:1805], GPP_03, markersize=4,color = :green, label = "")
rmse_value = rmse(GPP_03, y_sim_1_1_m)
text!(ax4, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[1,2]=ax4

In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp3, model_3, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end

xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis

ax5 = Axis(fig[2, 2], xlabel = "", ylabel = "", title="",xticks=(xvalues, xlabels))
CairoMakie.ylims!(ax5, -5, 15)

# Add shaded area
band!(ax5, time1[15:10:1805], ci[:, 1], ci[:, 2],  color = (:pink,1))
# Add mean line
lines!(ax5, time1[15:10:1805], GPP_03_be, color = :blue, linewidth = 4, label = "BE+noise")
lines!(ax5, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth =2, label = "mean")
GLMakie.scatter!(ax5, time1[15:10:1805], GPP_03,markersize=4, color = :green, label = "")
rmse_value = rmse(GPP_03, y_sim_1_1_m)
text!(ax5, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[2,2]=ax5


In [ ]:
y_sim_1_1 = simulate_observations(180, p_samp5, model_3, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end

xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis

ax6= Axis(fig[3, 2], xlabel =  "year", ylabel = "", title = "")
CairoMakie.ylims!(ax6, -5, 15)

# Add shaded area
band!(ax6, time1[15:10:1805], ci[:, 1], ci[:, 2], color = (:pink,1))
# Add mean line
lines!(ax6, time1[15:10:1805], GPP_03_be, color = :blue, linewidth = 4, label = "BE+noise")
lines!(ax6, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth =2, label = "mean")
GLMakie.scatter!(ax6, time1[15:10:1805], GPP_03,markersize=4, color = :green, label = "")
rmse_value = rmse(GPP_03, y_sim_1_1_m)
text!(ax6, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[3,2]=ax6


In [ ]:
function sample_observation(sample, model,N_PCA)  
    σ = sample[6] 
    β_1=sample[5]
    #β_0=sample[5]
    p = sample[1:4]
    θ = reshape(p, 1, length(p))
    m = model_5(θ,N_PCA)
    err=a5*randn(length(m))
    y = β_1*m +err
end

In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp1, model_5, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end
xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis

ax7 = Axis(fig[1,3], xlabel = "", ylabel = "", title = "US-NR1",xticks=(xvalues, xlabels))
CairoMakie.ylims!(ax7, -5, 15)
# Add shaded area
band!(ax7, time1[15:10:1805], ci[:, 1], ci[:, 2],  color = (:pink, 1))
# Add mean line
lines!(ax7, time1[15:10:1805], GPP_05_be, color =  :blue, linewidth = 4, label = "BE+noise")
lines!(ax7, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth = 2, label = "mean")
GLMakie.scatter!(ax7, time1[15:10:1805], GPP_05,markersize=4, color = :green, label = "")
rmse_value = rmse(GPP_05, y_sim_1_1_m)
text!(ax7, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[1,3]=ax7


In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp3, model_5, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end

xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis

ax8 = Axis(fig[2,3], xlabel = "", ylabel = "", title = "",xticks=(xvalues, xlabels))
CairoMakie.ylims!(ax8, -5, 15)

# Add shaded area
band!(ax8, time1[15:10:1805], ci[:, 1], ci[:, 2],  color = (:pink, 1))
# Add mean line
lines!(ax8, time1[15:10:1805], GPP_05_be, color = :blue, linewidth = 4, label = "BE+noise")
lines!(ax8, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth = 2, label = "mean")
GLMakie.scatter!(ax8, time1[15:10:1805], GPP_05,markersize=4, color = :green, label = "")
rmse_value = rmse(GPP_05, y_sim_1_1_m)
text!(ax8, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[2,3]=ax8

In [ ]:
# Simulate observations
y_sim_1_1 = simulate_observations(180, p_samp5, model_5, 12)
y_sim_1_1_m = mean(y_sim_1_1, dims=2)
ci = zeros(Float64, 180, 2)
for i = 1:size(y_sim_1_1, 1)
    ci[i, :] .= confidence_interval(y_sim_1_1[i, :])
end

xvalues = [2004, 2006, 2008]
xlabels = ["", "", ""]

# Create figure and axis

ax9 = Axis(fig[3, 3], xlabel = "year", ylabel = "",title="")
CairoMakie.ylims!(ax9, -5, 15)
# Add shaded area
band!(ax9, time1[15:10:1805], ci[:, 1], ci[:, 2],  color = (:pink,1))
# Add mean line
lines!(ax9, time1[15:10:1805], GPP_05_be, color = :blue, linewidth = 4, label = "BE+noise")
lines!(ax9, time1[15:10:1805],vec(y_sim_1_1_m), color = :red, linewidth = 2, label = "mean")
GLMakie.scatter!(ax9, time1[15:10:1805], GPP_05,markersize=4, color = :green, label = "")
rmse_value = rmse(GPP_05, y_sim_1_1_m)
text!(ax9, 2003, 13, text= string("NRMSE=",round(rmse_value, digits=2)), align = (:left, :center))
fig[3,3]=ax9

In [ ]:
Label(fig[1,2,Top()], "GPP at Predicted Sites (GP emulator)", fontsize=20, padding=(4, 4, 30, 4))
Label(fig[2,1,Left()], "Calibrated Sites (parameters)", rotation=π/2, fontsize=20, padding=(4, 60, 4, 4))
fig

In [ ]:
using CairoMakie
save("fig8_site_3_3_fluxnet_nobeta0_20250224.pdf",fig)